In [1]:
import os
import sys
import warnings
from dotenv import load_dotenv

warnings.filterwarnings("ignore")

In [2]:
load_dotenv()

True

In [7]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TRAVILY_API_KEY")

In [8]:
from tavily import TavilyClient

from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.chains import LLMChain

In [9]:
llm_model = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0.6,
    api_key=os.getenv("GROQ_API_KEY"))

In [10]:
messages = [
    SystemMessage("You are an advanced Legal AI Assistant designed to provide comprehensive legal insights, pattern recognition, and procedural guidance. Your knowledge base includes the Indian Penal Code (IPC), Code of Criminal Procedure (CrPC), Evidence Act, and regional laws."),
    HumanMessage("How do I deal with a situation where my neighbor is making too much noise at night?"),
]

print(type(llm_model.invoke(messages)))

<class 'langchain_core.messages.ai.AIMessage'>


In [32]:
tavily_client = TavilyClient(api_key=os.getenv("TRAVILY_API_KEY"))